# Delair.ai SDK Tutorial

In this notebook, you will be guided to :
- Create a **project**, **mission** and **survey**
- Create and upload **datasets** (image, rasters, point cloud, mesh, vector and file)
- Create an **annotation** and attach a graph to it

## ⚠️ Make sure to create a copy of this notebook before modifying it

To do that `File > Make a Copy...`

## Requirements

- Delair Python SDK : `pip install python-delairstack`

### You may execute this notebook one cell after the other with `Shift + Enter`

For more options, explore `Cell` in the top menu bar, or read a [tutorial](https://www.dataquest.io/blog/jupyter-notebook-tutorial/)

In [ ]:
import os
from delairstack import DelairStackSDK

In [ ]:
import getpass
platform_url = 'https://www.delair.ai'
login = input('Enter your email ')
password = getpass.getpass('Enter your password ')

In [ ]:
sdk = DelairStackSDK(url=platform_url, user=login, password=password)

### Download sample files (images, mesh, raster...)

An archive `Banana.zip` containing sample files will be downloaded (if not found in the current directory).

In [ ]:
import urllib.request
import zipfile

try: working_dir
except NameError: working_dir = os.getcwd()
%cd {working_dir}

if not os.path.exists('Banana'):
    print('"Banana" folder not found')
    if not os.path.exists('Banana.zip'):
        print('"Banana.zip" not found')
        print('Downloading it...', end=' ')
        url = 'https://delair-transfer.s3-eu-west-1.amazonaws.com/sdks/sample-data/Banana.zip'
        filename, _ = urllib.request.urlretrieve(url, 'Banana.zip')
        print('OK')

    print('Extracting "Banana.zip"...', end=' ')
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('.')
    print('OK')
else:
    print('"Banana" folder found. No need to download it again.')

In [ ]:
sample_path = './Banana'

In [ ]:
%cd {sample_path}
!ls .

# Create the project

In [ ]:
my_project = sdk.projects.create(
    name='SDK Tutorial',
    geometry={"coordinates": [[
                [12.397750168471589,-6.0021893390703696],
                [12.39799683152841,-6.0021893390703696],
                [12.39799683152841,-6.001515644167185],
                [12.397750168471589,-6.001515644167185],
                [12.397750168471589,-6.0021893390703696]]],
              "type": "Polygon"})

In [ ]:
print('We just created the project {!r} with id {!r}'.format(
    my_project.name, my_project.id))

### List the available project properties

In [ ]:
dir(my_project)

# Create the survey (mission + flight)

In [ ]:
my_flight, my_mission = sdk.missions.create_survey(
    name='My survey',
    coordinates=[
        [12.398218168471589,-6.002041339094632],
        [12.398506831528413,-6.002041339094632],
        [12.398506831528413,-6.0014106608166315],
        [12.398218168471589,-6.0014106608166315],
        [12.398218168471589,-6.002041339094632]],
    project=my_project.id,
    survey_date='2015-01-31T00:00:00.000Z',
    number_of_images=5)

In [ ]:
print('We just created the mission {!r} with id {!r}'.format(
    my_mission.name, my_mission.id))

# Create the datasets

## Image 1

In [ ]:
default_horizontal_srs_wkt = 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY\
["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],\
UNIT["degree",0.01745329251994328,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]'

In [ ]:
image1_dataset = sdk.datasets.create_image_dataset(
  name='00001499',
  project=my_project.id,
  mission=my_mission.id,
  flight=my_flight.id,
  geometry={"type": "Point",
            "coordinates": [12.397907, -6.001604983333333]},
  horizontal_srs_wkt=default_horizontal_srs_wkt,
  width=2448,
  height=2048
)

In [ ]:
sdk.datasets.upload_file(
  dataset=image1_dataset.id,
  component='image',
  file_path='00001499.jpg')

In [ ]:
print('Image dataset {!r} uploaded successfully'.format(image1_dataset.name))

### List the available dataset properties

In [ ]:
dir(image1_dataset)

## Same for the other images

In [ ]:
def create_and_upload_image_dataset(filename, coords):
    dataset_name = '.'.join(filename.split('.')[:-1])  # Name is filename without extension
    dataset = sdk.datasets.create_image_dataset(
        name=dataset_name,
        project=my_project.id,
        mission=my_mission.id,
        flight=my_flight.id,
        geometry={"type": "Point",
                  "coordinates": coords},
        horizontal_srs_wkt=default_horizontal_srs_wkt,
        width=2448,
        height=2048
    )

    sdk.datasets.upload_file(
      dataset=dataset.id,
      component='image',
      file_path=filename)
    return sdk.datasets.describe(dataset=dataset.id)

In [ ]:
image2_dataset = create_and_upload_image_dataset(filename='00001500.jpg', coords=[12.397885, -6.001729])
image3_dataset = create_and_upload_image_dataset(filename='00001501.jpg', coords=[12.397861, -6.001856])
image4_dataset = create_and_upload_image_dataset(filename='00001502.jpg', coords=[12.397843983333333, -6.001977])
image5_dataset = create_and_upload_image_dataset(filename='00001503.jpg', coords=[12.39784, -6.0021])

## Complete the survey upload
*Otherwise the upload progression will still be visible on the platform*

In [ ]:
sdk.missions.complete_survey_upload(flight=my_flight.id)

# Create the raster datasets

## Orthomosaic

In [ ]:
ortho_dataset = sdk.datasets.create_raster_dataset(
  name='Orthomosaic',
  project=my_project.id,
  mission=my_mission.id,
  dataset_format='geotiff',
  categories=['orthomosaic'])

sdk.datasets.upload_file(
  dataset=ortho_dataset.id,
  component='raster',
  file_path='Orthomosaic.tif')

## DSM (Digital Surface Model)

In [ ]:
dsm_dataset = sdk.datasets.create_raster_dataset(
  name='DSM',
  project=my_project.id,
  mission=my_mission.id,
  dataset_format='geotiff',
  categories=['dsm'])

sdk.datasets.upload_file(
  dataset=dsm_dataset.id,
  component='raster',
  file_path='DSM.tif')

# Create the point cloud dataset

In [ ]:
pcl_dataset = sdk.datasets.create_pcl_dataset(
  name='PointCloud',
  project=my_project.id,
  mission=my_mission.id,
  dataset_format='las')

sdk.datasets.upload_file(
  dataset=pcl_dataset.id,
  component='pcl',
  file_path='Point Cloud.las')

# Create the mesh dataset

In [ ]:
mesh_dataset = sdk.datasets.create_mesh_dataset(
  name='Mesh',
  project=my_project.id,
  mission=my_mission.id,
  dataset_format='obj',
  texture_count=1)

sdk.datasets.upload_file(
  dataset=mesh_dataset.id,
  component='mesh',
  file_path='Model.obj')

sdk.datasets.upload_file(
  dataset=mesh_dataset.id,
  component='material',
  file_path='Model.mtl')

sdk.datasets.upload_file(
  dataset=mesh_dataset.id,
  component='texture',
  file_path='Model_0.jpg')

# Create the vector dataset

In [ ]:
vector_dataset = sdk.datasets.create_vector_dataset(
    name='ContoursLines',
    project=my_project.id,
    mission=my_mission.id,
    dataset_format='geojson')

sdk.datasets.upload_file(
  dataset=vector_dataset.id,
  component='vector',
  file_path='Contours.geojson')

# Create an annotation

In [ ]:
my_annotation = sdk.annotations.create(
   project=my_project.id,
   mission=my_mission.id,
   geometry={
       "type": "Point",
       "coordinates": [12.397626065125216, -6.001589143208957]
   },
   name='New annotation',
   type='2d',
   icon=sdk.annotations.Icons.CONVEYOR
)

# The end

In [ ]:
from IPython.display import Markdown, display
from urllib.parse import urljoin

def printmd(string):
    display(Markdown(string))

printmd('### 💥 You can have a look at your project on the platform')
project_url = urljoin(platform_url, 'app/project/{}/view;leftPanel=layers'.format(my_project.id))
print(project_url)